<a href="https://colab.research.google.com/github/SamuelCdna/-predicciones-de-ventas-alimenticias-/blob/master/predicciones_de_ventas_alimenticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# predicciones-de-ventas-alimenticias

In [2]:
import pandas as pd
filename = "/content/sales_predictions.csv"

In [ ]:
df = pd.read_csv(filename) 
df.head() 